<a href="https://colab.research.google.com/github/alfa7g7/Analitica-prescriptiva/blob/main/Proyecto-final/Optimizacion/pf_optimizacion_ruta_soporte_colombia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROYECTO FINAL | ANALITICA PRESCRIPTIVA | PROYECTO OPTIMIZACION - PYTHON PULP

### INTEGRANTES:
              Arlex Pino
              Raul Echeverry
              Esteban Ordoñez
              Fabian Salazar Figueroa

---

##**Enunciado del Problema de Optimización: Asignación de Costo Mínimo para Soporte Técnico de ARL MARCA**

**1. Objetivo General:**
Determinar la asignación óptima de recursos de soporte técnico para minimizar el costo total mensual asociado a la prestación de servicios por parte de ARL MARCAA a sus clientes en todo el territorio colombiano, garantizando el cumplimiento de la demanda de servicio requerida en cada departamento.

**2. Contexto y Entidades:**

*   **Cobertura Geográfica:** El servicio debe poder ser asignado a cualquiera de los 32 departamentos de Colombia más Bogotá D.C. (total 33 destinos).
*   **Proveedores de Servicio:** Se cuenta con 10 empresas prestadoras externas (P1 a P10), cada una ubicada físicamente (sede) en uno de los 10 departamentos principales del país.
*   **Capacidad de los Proveedores:** Cada empresa prestadora `p` tiene una capacidad máxima definida de horas de trabajo disponibles mensualmente (`capacidad_horas[p]`).
*   **Recursos Humanos (Asesores):** Cada empresa prestadora dispone de 7 tipos diferentes de asesores de soporte (T1, T2, T3, T4, T5, T6, T7), clasificados por nivel de experticia, siendo T1 el más básico y T7 el más experto.

**3. Costos Involucrados:**

*   **Costo por Hora de Asesor:**
    *   El asesor tipo T1 tiene un costo base por hora.
    *   Los asesores T2 a T5 tienen un costo que incrementa un 25% respecto al tipo inmediatamente anterior.
    *   Los asesores T6 y T7 tienen un costo que incrementa un 40% respecto al tipo inmediatamente anterior (T5 y T6, respectivamente).
*   **Costos de Viaje:** Para asignar un asesor a un departamento distinto al de su sede, se incurre en costos adicionales:
    *   **Transporte Principal:** Un costo base por viaje (ida y regreso) que depende del par origen-destino y del modo (terrestre, aéreo, fluvial). *(Actualmente simulado)*.
    *   **Transporte Local:** Un costo fijo por viaje para movilización dentro del departamento destino.
    *   **Hospedaje:** Un costo fijo por noche de hospedaje, asumido por cada viaje interdepartamental.
    *   **Prorrateo:** El costo total fijo del viaje (transporte principal + local + hospedaje) se divide por un número promedio de horas de trabajo efectivas por viaje (supuesto: 6 horas) para obtener un costo de viaje por hora asignada. El costo total por hora para el modelo es la suma del costo del asesor y este costo de viaje prorrateado.

**4. Demanda de Servicio:**

*   **Demanda Granular:** Se define una demanda específica de horas mensuales para cada departamento destino `d` y para cada nivel *mínimo* de experticia requerido `t_req` (`demanda_dept_tipo[(d, t_req)]`). Esto significa que se sabe cuántas horas en `d` necesitan, como mínimo, un asesor T1, cuántas necesitan al menos un T2, y así sucesivamente hasta T7. *(Actualmente simulada, distribuyendo una demanda total departamental simulada mediante porcentajes base por tipo)*.

**5. Reglas Operativas y Restricciones:**

*   **Sustitución de Asesores:** Se asume que un asesor de un nivel de experticia superior `t` puede satisfacer la demanda que requiere un nivel de experticia igual o inferior `t_req` (es decir, si `nivel(t) >= nivel(t_req)`).
*   **Satisfacción de Demanda (Restricción Clave):** Para cada departamento `d` y cada tipo de demanda `t_req`, la suma total de horas asignadas a ese departamento por *todos* los asesores `t` que estén *calificados* (según la regla de sustitución) debe ser mayor o igual a la demanda específica `demanda_dept_tipo[(d, t_req)]`.
*   **Límite de Capacidad por Proveedor:** La suma total de horas asignadas *por* una empresa prestadora `p` (sumando sobre todos los tipos de asesor que asigna y todos los destinos a los que los envía) no puede exceder su capacidad mensual total `capacidad_horas[p]`.
*   **Límite de Composición de Fuerza Laboral (70/30):** Para cada empresa prestadora `p`, las horas *efectivamente asignadas* utilizando asesores de tipo básico (T1 a T4) no pueden superar el 70% de su capacidad total `capacidad_horas[p]`. De forma análoga, las horas asignadas utilizando asesores de tipo experto (T5 a T7) no pueden superar el 30% de su capacidad total `capacidad_horas[p]`.

**6. Decisión a Optimizar:**
El modelo debe determinar el valor de `horas_asignadas[p, t, d]`: la cantidad de horas que la empresa prestadora `p` debe asignar utilizando un asesor de tipo `t` para atender la demanda en el departamento destino `d`, de forma que se cumplan todas las restricciones y se minimice el costo total mensual (suma de `horas_asignadas * costo_total_por_hora`).

**7. Supuestos Clave Actuales (para Contexto):**
*   Costos de transporte principal entre departamentos son simulados.
*   La demanda granular (por departamento y tipo) es simulada.
*   Los costos fijos de viaje se prorratean linealmente por hora asignada.
*   La sustitución de asesores por niveles superiores es perfecta (sin pérdida de eficiencia).
*   Las horas asignadas pueden ser fraccionarias (modelo continuo).

---

###**1. Descripción del Problema de Optimización**

El objetivo de este modelo es **minimizar el costo total mensual** incurrido por ARL MARCA para satisfacer las necesidades de soporte técnico en todos los departamentos de Colombia. Este soporte es proporcionado por 10 empresas prestadoras externas, cada una con una sede en un departamento principal y una capacidad máxima de horas de trabajo mensuales.

Cada prestadora dispone de 7 tipos de asesores (T1 a T7), con costos por hora incrementales, siendo T1 el más básico y T7 el más experto. El modelo debe determinar **cuántas horas de cada tipo de asesor (`t`) debe asignar cada empresa prestadora (`p`) a cada departamento destino (`d`)** para cumplir con la demanda específica.

La **demanda es granular**, es decir, se especifica para cada departamento (`d`) cuántas horas requieren un nivel de habilidad *mínimo* correspondiente a cada tipo de asesor (`t_req`). Se asume que un asesor de nivel superior (`t`) **puede satisfacer** una demanda de nivel igual o inferior (`t_req <= t`).

El costo total a minimizar incluye:

1.  **Costo por Hora de Asesor:** Según el tipo de asesor asignado.
2.  **Costos de Viaje (prorrateados por hora):** Incluyen transporte principal (terrestre, aéreo, fluvial simulado), transporte local en destino y hospedaje (si aplica), distribuidos sobre una cantidad promedio de horas trabajadas por viaje.

El modelo debe respetar las siguientes **restricciones clave**:

*   **Satisfacción de Demanda Granular:** La suma de horas proporcionadas por asesores calificados (nivel `t >= t_req`) a un departamento `d` debe ser mayor o igual a la demanda específica para el nivel `t_req` en ese departamento.
*   **Capacidad Máxima por Prestadora:** Las horas totales asignadas *por* una prestadora no pueden exceder su capacidad mensual.
*   **Límites de Uso de Recursos (70/30):** Las horas totales asignadas *por* una prestadora utilizando asesores básicos (T1-T4) no pueden exceder el 70% de su capacidad total. Similarmente, las horas asignadas utilizando asesores expertos (T5-T7) no pueden exceder el 30% de su capacidad total. Estos límites reflejan restricciones en la composición de la fuerza laboral efectivamente utilizada.

---

###**2. Datos y Supuestos**

**Datos Necesarios:**

*   **Lista de Departamentos:** 33 destinos (32 departamentos + Bogotá D.C.).
*   **Lista de Empresas Prestadoras:** 10 entidades (P1 a P10).
*   **Sedes de Prestadoras:** Ubicación (departamento) de la sede principal de cada prestadora.
*   **Tipos de Asesor:** 7 niveles (T1 a T7).
*   **Costo por Hora Asesor:** Calculado según la regla incremental (base +25% T2-T5, +40% T6-T7).
*   **Capacidad Máxima de Horas:** Límite mensual de horas por cada empresa prestadora.
*   **Costos Fijos de Viaje:** Tarifas base para transporte terrestre, aéreo, fluvial; costo de transporte local; costo de hospedaje por noche.
*   **Demanda Granular (`demanda_dept_tipo[(d, t_req)]`):** Número de horas requeridas en el departamento `d` que necesitan *al menos* un asesor de nivel `t_req`. **(Actualmente SIMULADA basada en porcentajes por tipo sobre una demanda total departamental también simulada).**

**Supuestos Clave del Modelo Actual:**

1.  **Costos de Viaje Simulados:** La matriz de costos de transporte base entre sedes y destinos es **simulada**. Se requiere información real para resultados precisos.
2.  **Prorrateo de Costos de Viaje:** Los costos fijos de viaje se dividen por un número **promedio de horas por viaje** (supuesto: 6 horas) para obtener un costo adicional por hora asignada. No modela viajes discretos.
3.  **Demanda Simulada:** Tanto la demanda total por departamento como su distribución granular por tipo de asesor requerido son **simuladas** con porcentajes fijos. Se necesitan datos reales de demanda.
4.  **Sustitución de Asesores:** Se asume que un asesor de nivel superior **puede realizar el trabajo** de cualquier nivel inferior sin penalización de eficiencia o costo adicional (más allá de su propia tarifa horaria).
5.  **Límites 70/30 como Techo de Uso:** Las restricciones del 70% (T1-T4) y 30% (T5-T7) actúan como un **límite superior** sobre las horas *efectivamente asignadas* por cada prestadora, basado en su capacidad total. No dictan la composición exacta si la demanda no requiere usar toda la capacidad.
6.  **Variables Continuas:** Las `horas_asignadas` pueden tomar valores decimales.
7.  **Linealidad:** Se asume que todos los costos son lineales (sin descuentos por volumen, etc.).
8.  **Disponibilidad Implícita:** Se asume que si una prestadora tiene capacidad en un grupo (ej. básicos), puede asignar cualquier tipo dentro de ese grupo (ej. T1, T2, T3, o T4) hasta agotar el límite del 70%, sujeto a la demanda. No considera un número fijo de asesores específicos de cada tipo por prestadora.


---

### **3. Modelo**

In [ ]:
# -*- coding: utf-8 -*-

# Instalar PuLP en Colab
!pip install pulp

# Librerias a usar
import pulp
import pandas as pd
import numpy as np
import itertools # Para iterar sobre combinaciones d, t

In [ ]:
#----------------------------------------------------
# 1. Definición de Datos y Parámetros
#----------------------------------------------------

# Departamentos (Destinos) - Lista completa (32 + Bogotá)
# Fuente: DANE (puede variar ligeramente si se agrupan áreas no municipalizadas)
departamentos = [
    'Amazonas', 'Antioquia', 'Arauca', 'Atlántico', 'Bogotá D.C.', 'Bolívar',
    'Boyacá', 'Caldas', 'Caquetá', 'Casanare', 'Cauca', 'Cesar', 'Chocó',
    'Córdoba', 'Cundinamarca', 'Guainía', 'Guaviare', 'Huila', 'La Guajira',
    'Magdalena', 'Meta', 'Nariño', 'Norte de Santander', 'Putumayo', 'Quindío',
    'Risaralda', 'San Andrés y Providencia', 'Santander', 'Sucre', 'Tolima',
    'Valle del Cauca', 'Vaupés', 'Vichada'
]
num_departamentos = len(departamentos)

# Empresas Prestadoras (Orígenes)
empresas_prestadoras = [f'P{i}' for i in range(1, 11)]
num_prestadoras = len(empresas_prestadoras)

# Tipos de Asesor
tipos_asesor = [f'T{i}' for i in range(1, 8)]
num_tipos_asesor = len(tipos_asesor)

# Sede de cada Empresa Prestadora (SUPUESTO - Asignar a dptos principales)
# Necesitarías los datos reales. Ejemplo:
sedes_prestadoras = {
    'P1': 'Antioquia',
    'P2': 'Atlántico',
    'P3': 'Bogotá D.C.',
    'P4': 'Valle del Cauca',
    'P5': 'Santander',
    'P6': 'Cundinamarca', # Sede diferente a Bogotá
    'P7': 'Bolívar',
    'P8': 'Boyacá',
    'P9': 'Nariño',
    'P10': 'Meta'
}

# Capacidad Mensual de Horas por Prestadora
capacidad_horas = {
    'P1': 6000, 'P2': 5000, 'P3': 7000, 'P4': 10000, 'P5': 12000,
    'P6': 8000, 'P7': 6000, 'P8': 6000, 'P9': 2000, 'P10': 4000
}

tipos_basicos = ['T1', 'T2', 'T3', 'T4']
tipos_expertos = ['T5', 'T6', 'T7']

# Helper para comparar niveles de asesor
tipo_to_int = {tipo: i for i, tipo in enumerate(tipos_asesor, 1)} # T1->1, T2->2, ... T7->7

# Costo Base por Hora Asesor Tipo 1
costo_base_t1 = 100000

# Calcular Costo por Hora para cada Tipo de Asesor
costo_hora_asesor = {}
costo_hora_asesor['T1'] = costo_base_t1
costo_hora_asesor['T2'] = costo_hora_asesor['T1'] * 1.25
costo_hora_asesor['T3'] = costo_hora_asesor['T2'] * 1.25
costo_hora_asesor['T4'] = costo_hora_asesor['T3'] * 1.25
costo_hora_asesor['T5'] = costo_hora_asesor['T4'] * 1.25
costo_hora_asesor['T6'] = costo_hora_asesor['T5'] * 1.40 # Incremento del 40%
costo_hora_asesor['T7'] = costo_hora_asesor['T6'] * 1.40 # Incremento del 40%

print("Costos por hora por tipo de asesor:")
for tipo, costo in costo_hora_asesor.items():
    print(f"{tipo}: ${costo:,.2f}")
print("-" * 30)

# Costos Fijos de Viaje
costo_transporte_terrestre = 120000
costo_transporte_aereo = 360000
costo_transporte_fluvial = 240000
costo_transporte_local = 100000
costo_hospedaje_noche = 80000 # Asumimos 1 noche por viaje que requiera desplazamiento interdepartamental

# SUPUESTO: Horas promedio de trabajo efectivo por viaje
horas_promedio_por_viaje = 6

# ----------------------------------------------------
# SUPUESTO CRÍTICO: Matriz de Costo Base de Transporte (Origen-Destino)
# ----------------------------------------------------
# Esto DEBE ser reemplazado con datos reales.
# Generaremos una matriz simulada simple:
# - 0 si origen == destino
# - Costo terrestre por defecto si no son Amazonas, San Andrés, Chocó, Guainía, Vaupés, Vichada (requieren aéreo/fluvial)
# - Costo aéreo/fluvial para esos departamentos especiales o si son muy distantes (simulado)

np.random.seed(42) # Para reproducibilidad
costo_base_transporte = pd.DataFrame(index=departamentos, columns=departamentos, dtype=float)

departamentos_especiales = ['Amazonas', 'San Andrés y Providencia', 'Chocó', 'Guainía', 'Vaupés', 'Vichada', 'Putumayo', 'Guaviare', 'Caquetá']

for origen in departamentos:
    for destino in departamentos:
        if origen == destino:
            costo_base_transporte.loc[origen, destino] = 0
        # Simulación simple: aéreo/fluvial a dptos especiales o si 'lejanos' (aleatorio simple)
        elif origen in departamentos_especiales or destino in departamentos_especiales or np.random.rand() > 0.85:
             # Asignar aleatoriamente entre aéreo y fluvial (o el más caro si es San Andrés)
             if destino == 'San Andrés y Providencia' or origen == 'San Andrés y Providencia':
                 costo_base_transporte.loc[origen, destino] = costo_transporte_aereo
             else:
                 costo_base_transporte.loc[origen, destino] = np.random.choice([costo_transporte_aereo, costo_transporte_fluvial])
        else:
            # Terrestre para la mayoría de los casos continentales no especiales
             costo_base_transporte.loc[origen, destino] = costo_transporte_terrestre

print("Ejemplo Costos Base Transporte (Simulado):")
print(costo_base_transporte.loc['Antioquia', ['Bogotá D.C.', 'Amazonas', 'San Andrés y Providencia']])
print("-" * 30)

# Calcular Costo Total por Viaje (incluyendo local y hospedaje si hay desplazamiento)
costo_total_viaje = pd.DataFrame(index=empresas_prestadoras, columns=departamentos, dtype=float)
for p in empresas_prestadoras:
    sede_origen = sedes_prestadoras[p]
    for d in departamentos:
        costo_base = costo_base_transporte.loc[sede_origen, d]
        if costo_base > 0: # Si hay viaje interdepartamental
            costo_total_viaje.loc[p, d] = costo_base + costo_transporte_local + costo_hospedaje_noche
        else: # Viaje dentro del mismo departamento (asumimos costo 0 de viaje principal y hospedaje)
            # Podríamos incluir costo local aquí si siempre aplica, pero lo dejaremos en 0 por simplicidad
            # ya que el enunciado no lo aclara para viajes locales. Si se requiere, sumar costo_transporte_local.
            costo_total_viaje.loc[p, d] = 0

# Calcular Costo de Viaje por Hora Asignada
costo_viaje_por_hora = costo_total_viaje / horas_promedio_por_viaje

# Calcular Costo Total por Hora (Asesor + Viaje prorrateado)
costo_total_por_hora = {}
for p in empresas_prestadoras:
    for t in tipos_asesor:
        for d in departamentos:
            costo_total_por_hora[(p, t, d)] = costo_hora_asesor[t] + costo_viaje_por_hora.loc[p, d]


Costos por hora por tipo de asesor:
T1: $100,000.00
T2: $125,000.00
T3: $156,250.00
T4: $195,312.50
T5: $244,140.62
T6: $341,796.88
T7: $478,515.62
------------------------------
Ejemplo Costos Base Transporte (Simulado):
Bogotá D.C.                 120000.0
Amazonas                    360000.0
San Andrés y Providencia    360000.0
Name: Antioquia, dtype: float64
------------------------------


In [ ]:
# ----------------------------------------------------
# Simulación de Demanda Granular (por Departamento y Tipo)
# ----------------------------------------------------
np.random.seed(123) # Semilla para reproducibilidad
demanda_base_total_dept = 50 # Horas base TOTALES demandadas en cada departamento
demanda_extra_principales = 200 # Horas extra TOTALES para dptos clave

# 1. Calcular Demanda Total por Departamento
demanda_total_dept = {d: demanda_base_total_dept + np.random.randint(0, 50) for d in departamentos}
dptos_principales_demanda = ['Antioquia', 'Atlántico', 'Bogotá D.C.', 'Valle del Cauca', 'Santander', 'Cundinamarca']
for d in dptos_principales_demanda:
    if d in demanda_total_dept:
        demanda_total_dept[d] += demanda_extra_principales + np.random.randint(0, 100)

# 2. Distribuir la demanda total de cada depto entre los tipos de asesor
#    Usaremos porcentajes base, con ligera variación aleatoria por departamento.
#    Estos porcentajes representan la proporción de la demanda que requiere *al menos* ese nivel.
#    ¡OJO! La suma de estos porcentajes puede ser > 100% si no se ajusta bien.
#    Mejor enfoque: Definir porcentajes para *exactamente* qué tipo se requiere.

base_pct_demanda_tipo = { # Porcentaje de la demanda total de un depto que requiere ESTE tipo específico
    'T1': 0.15, 'T2': 0.20, 'T3': 0.20, 'T4': 0.15, # Suma 70% básicos
    'T5': 0.10, 'T6': 0.10, 'T7': 0.10  # Suma 30% expertos
}
# Asegurar que suma 1 (o muy cerca)
assert abs(sum(base_pct_demanda_tipo.values()) - 1.0) < 1e-6, "Los porcentajes base deben sumar 1"

demanda_dept_tipo = {} # Diccionario con clave (d, t) -> horas requeridas de tipo t en depto d
for d in departamentos:
    total_d = demanda_total_dept[d]
    # Aplicar porcentajes base para distribuir la demanda total del departamento
    for t in tipos_asesor:
        # Podríamos añadir una pequeña aleatoriedad aquí a los porcentajes por depto,
        # pero por simplicidad, usamos los base por ahora.
        horas_requeridas = round(total_d * base_pct_demanda_tipo[t])
        if horas_requeridas > 0:
            demanda_dept_tipo[(d, t)] = horas_requeridas
    # Opcional: Ajustar redondeos para que la suma por tipo iguale (o casi) la demanda total del dpto.
    # Por simplicidad, lo dejamos así por ahora.

print("\nDemanda Mensual de Horas por Departamento y Tipo (Simulada):")
# Imprimir algunos ejemplos
print("Ejemplos Demanda Granular:")
for key in list(demanda_dept_tipo.keys())[:10]:
     print(f"  {key}: {demanda_dept_tipo[key]} horas")
# Verificar suma por un departamento ejemplo
d_ejemplo = 'Antioquia'
suma_granular_ejemplo = sum(demanda_dept_tipo.get((d_ejemplo, t), 0) for t in tipos_asesor)
print(f"\nDemanda total simulada para {d_ejemplo}: {demanda_total_dept.get(d_ejemplo, 0)}")
print(f"Suma de demanda granular simulada para {d_ejemplo}: {suma_granular_ejemplo}")
print("-" * 30)


# Comprobación de factibilidad básica: Total Demanda vs Total Capacidad
total_demanda_simulada = sum(demanda_dept_tipo.values()) # Suma de toda la demanda granular
total_capacidad_oferta = sum(capacidad_horas.values())
print(f"Total Horas Demandadas (Granular Simulado): {total_demanda_simulada:,.0f}")
print(f"Total Horas Ofertadas (Capacidad): {total_capacidad_oferta:,.0f}")
if total_demanda_simulada > total_capacidad_oferta:
    print("\n*** ADVERTENCIA: La demanda total simulada excede la capacidad total.")
    print("*** El modelo probablemente será INFACTIBLE.")
else:
    print("\nLa capacidad total es suficiente para cubrir la demanda total simulada.")
print("-" * 30)


Demanda Mensual de Horas por Departamento y Tipo (Simulada):
Ejemplos Demanda Granular:
  ('Amazonas', 'T1'): 14 horas
  ('Amazonas', 'T2'): 19 horas
  ('Amazonas', 'T3'): 19 horas
  ('Amazonas', 'T4'): 14 horas
  ('Amazonas', 'T5'): 10 horas
  ('Amazonas', 'T6'): 10 horas
  ('Amazonas', 'T7'): 10 horas
  ('Antioquia', 'T1'): 47 horas
  ('Antioquia', 'T2'): 63 horas
  ('Antioquia', 'T3'): 63 horas

Demanda total simulada para Antioquia: 313
Suma de demanda granular simulada para Antioquia: 313
------------------------------
Total Horas Demandadas (Granular Simulado): 4,110
Total Horas Ofertadas (Capacidad): 66,000

La capacidad total es suficiente para cubrir la demanda total simulada.
------------------------------


In [ ]:
#----------------------------------------------------
# 2. Creación del Modelo de Optimización
#----------------------------------------------------
# Crear el problema de minimización
model = pulp.LpProblem("Optimizacion_Costos_Soporte_ARL_Demanda_Granular", pulp.LpMinimize)

In [ ]:
#----------------------------------------------------
# 3. Definición de Variables de Decisión
#----------------------------------------------------
# H(p, t, d): Número de horas asignadas desde la prestadora 'p',
#             con un asesor tipo 't', al departamento destino 'd'.
# Usaremos variables continuas por simplicidad, podrían ser enteras (LpInteger) si se requiere precisión absoluta de horas.
horas_asignadas = pulp.LpVariable.dicts("Horas",
                                        ((p, t, d) for p in empresas_prestadoras for t in tipos_asesor for d in departamentos),
                                        lowBound=0,
                                        cat='Continuous') # o 'Integer'


In [ ]:
#----------------------------------------------------
# 4. Definición de la Función Objetivo
#----------------------------------------------------
# Minimizar el costo total = suma (horas_asignadas * costo_total_por_hora)
model += pulp.lpSum(horas_asignadas[p, t, d] * costo_total_por_hora[(p, t, d)]
                    for p in empresas_prestadoras
                    for t in tipos_asesor
                    for d in departamentos), "Costo_Total_Satisfacer_Demanda_Granular"


In [ ]:
#----------------------------------------------------
# 5. Definición de las Restricciones
#----------------------------------------------------
"""
# R1: Capacidad Total de Horas por Prestadora (Asignar todas las horas disponibles - según supuesto)
for p in empresas_prestadoras:
    model += pulp.lpSum(horas_asignadas[p, t, d] for t in tipos_asesor for d in departamentos) == capacidad_horas[p], f"Capacidad_Total_{p}"

# R2: Distribución de Horas por Tipo de Asesor dentro de cada Prestadora
tipos_basicos = ['T1', 'T2', 'T3', 'T4']
tipos_expertos = ['T5', 'T6', 'T7']

for p in empresas_prestadoras:
    # 70% para tipos básicos
    model += pulp.lpSum(horas_asignadas[p, t, d] for t in tipos_basicos for d in departamentos) == 0.70 * capacidad_horas[p], f"Distribucion_Basicos_{p}"
    # 30% para tipos expertos
    model += pulp.lpSum(horas_asignadas[p, t, d] for t in tipos_expertos for d in departamentos) == 0.30 * capacidad_horas[p], f"Distribucion_Expertos_{p}"

# (Opcional) R3: Asegurar Cobertura Mínima por Departamento (si no se asume asignar todo)
# Si en lugar de asignar TODAS las horas, quisiéramos asegurar una demanda mínima por departamento 'd':
# demanda_minima = 1 # Hora simbólica, o un valor real si se conoce
# for d in departamentos:
#    model += pulp.lpSum(horas_asignadas[p, t, d] for p in empresas_prestadoras for t in tipos_asesor) >= demanda_minima, f"Cobertura_Minima_{d}"
# Nota: Esta restricción entraría en conflicto con la R1 y R2 si la suma de demandas mínimas excede la capacidad total. Elegir una estrategia: o asignar todo (como está ahora) o satisfacer demanda (requiere datos de demanda).
"""

# MODIFICACIÓN R1: Capacidad Total de Horas por Prestadora (Límite Superior)
for p in empresas_prestadoras:
    # La suma de horas asignadas por 'p' no puede exceder su capacidad
    model += pulp.lpSum(horas_asignadas[p, t, d] for t in tipos_asesor for d in departamentos) <= capacidad_horas[p], f"Capacidad_Max_Total_{p}"

# MODIFICACIÓN R2: Límite de Horas por Grupo de Tipo de Asesor (70/30 de la Capacidad)
for p in empresas_prestadoras:
    # Las horas asignadas de T1-T4 no pueden exceder el 70% de la capacidad TOTAL de 'p'
    model += pulp.lpSum(horas_asignadas[p, t, d] for t in tipos_basicos for d in departamentos) <= 0.70 * capacidad_horas[p], f"Limite_Capacidad_Basicos_{p}"
    # Las horas asignadas de T5-T7 no pueden exceder el 30% de la capacidad TOTAL de 'p'
    model += pulp.lpSum(horas_asignadas[p, t, d] for t in tipos_expertos for d in departamentos) <= 0.30 * capacidad_horas[p], f"Limite_Capacidad_Expertos_{p}"

# MODIFICACIÓN R3: Satisfacción de la Demanda GRÁNULAR por Departamento y Tipo
# ASUNCIÓN: Un asesor de tipo 't' puede satisfacer demanda de tipo 't_req' si tipo_to_int[t] >= tipo_to_int[t_req]
for d in departamentos:
    for t_req in tipos_asesor: # t_req es el tipo de demanda que necesitamos satisfacer
        demanda_especifica = demanda_dept_tipo.get((d, t_req), 0) # Obtenemos la demanda para este dpto y tipo

        if demanda_especifica > 0: # Solo añadir restricción si hay demanda de este tipo
            # La suma de horas asignadas por prestadoras (p) usando asesores (t)
            # que estén *calificados* para la tarea requerida (t_req),
            # debe ser >= a la demanda específica.
            model += pulp.lpSum(horas_asignadas[p, t, d]
                                for p in empresas_prestadoras
                                for t in tipos_asesor
                                if tipo_to_int[t] >= tipo_to_int[t_req] # Condición de calificación
                               ) >= demanda_especifica, f"Satisfacer_Demanda_{d}_{t_req}"

In [ ]:
#----------------------------------------------------
# 6. Solución del Modelo
#----------------------------------------------------
"""
print("\nResolviendo el modelo...")
model.solve()
"""
print("\nResolviendo el modelo con demanda granular...")
solver = pulp.PULP_CBC_CMD(msg=1) # Añadir msg=1 para ver más output del solver si tarda o falla
model.solve(solver)


Resolviendo el modelo con demanda granular...


1

In [ ]:
#----------------------------------------------------
# 7. Presentación de Resultados
#----------------------------------------------------
print("-" * 30)
print(f"Estado de la Solución: {pulp.LpStatus[model.status]}")

if pulp.LpStatus[model.status] == 'Optimal':
    print(f"Costo Total Óptimo Mensual para Satisfacer Demanda Granular: ${model.objective.value():,.2f}")
    print("-" * 30)
    print("Resumen de Horas Asignadas (ejemplo):")

    resultados = []
    for p in empresas_prestadoras:
        for t in tipos_asesor:
            for d in departamentos:
                horas = horas_asignadas[p, t, d].varValue
                if horas > 0.01: # Usar umbral pequeño para capturar asignaciones
                    resultados.append({
                        'Prestadora': p,
                        'Sede': sedes_prestadoras[p],
                        'Tipo Asesor Asignado': t, # El tipo que se envió
                        'Departamento Destino': d,
                        'Horas Asignadas': horas,
                        #'Demanda Dept': demanda_dept[d], # Añadir demanda para referencia
                        #'Demanda Requerida (Tipo)': demanda_dept_tipo.get((d, ???)), # Difícil mostrar aquí qué demanda específica cubrió
                        'Costo Total x Hora': costo_total_por_hora[(p, t, d)],
                        'Costo Parcial': horas * costo_total_por_hora[(p, t, d)]
                    })

    df_resultados = pd.DataFrame(resultados)
    df_resultados = df_resultados.sort_values(by=['Departamento Destino', 'Prestadora', 'Tipo Asesor Asignado'])

    print(df_resultados.head(20))
    # df_resultados.to_csv('asignacion_optima_demanda.csv', index=False)
    # print("\nResultados completos guardados en 'asignacion_optima_demanda.csv'")

    # --- Verificaciones Actualizadas ---

    print("\nVerificación de Cumplimiento de Demanda Granular por Departamento y Tipo:")
    cumplimiento_demanda = True
    detalles_incumplimiento = []
    for d in departamentos:
        for t_req in tipos_asesor:
            demanda_req = demanda_dept_tipo.get((d, t_req), 0)
            if demanda_req > 0:
                # Sumar las horas asignadas por asesores calificados para esta demanda específica
                horas_provistas_calificadas = sum(horas_asignadas[p, t, d].varValue
                                                  for p in empresas_prestadoras
                                                  for t in tipos_asesor
                                                  if tipo_to_int[t] >= tipo_to_int[t_req] and horas_asignadas[p, t, d].varValue is not None)

                if horas_provistas_calificadas < demanda_req - 0.01: # Tolerancia numérica
                    cumplimiento_demanda = False
                    detalles_incumplimiento.append(f" Depto: {d}, Tipo Req: {t_req}, Demanda: {demanda_req:.2f}, Provisto: {horas_provistas_calificadas:.2f}")

    if cumplimiento_demanda:
        print("¡Todas las demandas granulares fueron satisfechas!")
    else:
        print("¡ERROR! Algunas demandas granulares NO fueron satisfechas:")
        for detalle in detalles_incumplimiento[:10]: # Mostrar solo los primeros 10 errores
            print(detalle)

    # (Las verificaciones de Capacidad Total y Límites 70/30
    print("\nVerificación de Uso de Capacidad por Prestadora (Asignado <= Capacidad):")
    horas_asignadas_prestadora = df_resultados.groupby('Prestadora')['Horas Asignadas'].sum()
    df_capacidad_check = pd.DataFrame({'Capacidad': capacidad_horas, 'Horas Asignadas': horas_asignadas_prestadora}).fillna(0)
    df_capacidad_check['% Uso Capacidad'] = (df_capacidad_check['Horas Asignadas'] / df_capacidad_check['Capacidad']) * 100
    print(df_capacidad_check)


    print("\nVerificación de Límites 70/30 por Prestadora (Asignado <= Límite):")
    # Usamos df_resultados que tiene las horas asignadas por PRESTADORA y TIPO ASIGNADO
    distribucion = df_resultados.groupby(['Prestadora', 'Tipo Asesor Asignado'])['Horas Asignadas'].sum().unstack(fill_value=0)
    distribucion = distribucion.reindex(columns=tipos_asesor, fill_value=0)

    distribucion['Asignado_Basicos'] = distribucion[tipos_basicos].sum(axis=1)
    distribucion['Asignado_Expertos'] = distribucion[tipos_expertos].sum(axis=1)
    distribucion['Limite_Basicos (70%)'] = [0.70 * capacidad_horas[p] for p in distribucion.index]
    distribucion['Limite_Expertos (30%)'] = [0.30 * capacidad_horas[p] for p in distribucion.index]

    distribucion['Cumple_Limite_Basicos'] = distribucion['Asignado_Basicos'] <= distribucion['Limite_Basicos (70%)'] + 0.01
    distribucion['Cumple_Limite_Expertos'] = distribucion['Asignado_Expertos'] <= distribucion['Limite_Expertos (30%)'] + 0.01

    print(distribucion[['Asignado_Basicos', 'Limite_Basicos (70%)', 'Cumple_Limite_Basicos',
                        'Asignado_Expertos', 'Limite_Expertos (30%)', 'Cumple_Limite_Expertos']])
    print(f"Todas las prestadoras cumplen límite 70%? {distribucion['Cumple_Limite_Basicos'].all()}")
    print(f"Todas las prestadoras cumplen límite 30%? {distribucion['Cumple_Limite_Expertos'].all()}")


elif pulp.LpStatus[model.status] == 'Infeasible':
    print("El modelo no tiene una solución factible con demanda granular.")
    print("Posibles causas:")
    print("- Demanda Total > Capacidad Total.")
    print("- Demanda de tipos específicos (ej. muchos expertos) supera la capacidad disponible para esos tipos (considerando el límite 70/30 y la sustituibilidad).")
    print("- Combinación de altos costos de viaje y demanda en lugares remotos que hacen imposible/ineficiente cumplirla.")
    print("- Revisa la simulación de demanda granular, las capacidades y los límites 70/30.")

else:
    print(f"No se encontró una solución óptima. Estado: {pulp.LpStatus[model.status]}")

------------------------------
Estado de la Solución: Optimal
Costo Total Óptimo Mensual para Satisfacer Demanda Granular: $290,623,437.50
------------------------------
Resumen de Horas Asignadas (ejemplo):
   Prestadora         Sede Tipo Asesor Asignado Departamento Destino  \
95        P10         Meta                   T4             Amazonas   
97        P10         Meta                   T7             Amazonas   
77         P9       Nariño                   T3             Amazonas   
0          P1    Antioquia                   T3            Antioquia   
2          P1    Antioquia                   T4            Antioquia   
5          P1    Antioquia                   T7            Antioquia   
6          P1    Antioquia                   T7               Arauca   
56         P7      Bolívar                   T3               Arauca   
60         P7      Bolívar                   T4               Arauca   
10         P2    Atlántico                   T3            Atlántico   


---

### **4. Explicación del Código y Modelo (Versión con Demanda Granular)**

1.  **Setup:** Importación de librerías (`pulp`, `pandas`, `numpy`, `itertools`).
2.  **Datos y Parámetros:**
    *   Definición de listas/diccionarios para departamentos, prestadoras, tipos, sedes, capacidades.
    *   Cálculo de `costo_hora_asesor` para cada tipo.
    *   Definición de costos fijos de viaje.
    *   **Simulación de Costos de Viaje:** Creación de `costo_base_transporte` (simulada), cálculo de `costo_total_viaje` (base+local+hospedaje si aplica) y `costo_viaje_por_hora` (prorrateado).
    *   Cálculo de `costo_total_por_hora[(p, t, d)]` (costo asesor + costo viaje prorrateado).
    *   **Simulación de Demanda Granular:** Se simula una `demanda_total_dept`, luego se distribuye usando `base_pct_demanda_tipo` para crear `demanda_dept_tipo[(d, t_req)]`, las horas requeridas en `d` de nivel mínimo `t_req`.
    *   Se crea el helper `tipo_to_int` para comparar niveles de asesor.
    *   Se realiza una verificación básica de factibilidad (demanda total simulada vs capacidad total).
3.  **Creación del Modelo:** `pulp.LpProblem` configurado para minimización (`LpMinimize`).
4.  **Variables de Decisión:** `horas_asignadas[p, t, d]` representa las horas asignadas por `p` con tipo `t` a `d` (continuas, >= 0).
5.  **Función Objetivo:** Minimizar la suma de `horas_asignadas[p, t, d] * costo_total_por_hora[(p, t, d)]` sobre todas las combinaciones. Busca el costo mínimo para satisfacer la demanda granular.
6.  **Restricciones:**
    *   **R1 - Capacidad Máxima por Prestadora (`<=`):** Suma de `horas_asignadas` por `p` (sobre todos `t`, `d`) `<= capacidad_horas[p]`.
    *   **R2 - Límites de Uso 70/30 (`<=`):** Suma de `horas_asignadas` por `p` usando tipos básicos (T1-T4) `<= 0.70 * capacidad_horas[p]`. Análogo para tipos expertos (T5-T7) con `<= 0.30 * capacidad_horas[p]`.
    *   **R3 - Satisfacción de Demanda Granular (`>=`):** Para cada `d` y cada tipo requerido `t_req` con `demanda_especifica > 0`: la suma de `horas_asignadas[p, t, d]` sobre todas las prestadoras `p` y todos los tipos de asesor asignado `t` **tales que `t` esté calificado** (`tipo_to_int[t] >= tipo_to_int[t_req]`) debe ser `>= demanda_especifica[(d, t_req)]`. Esta es la restricción clave que implementa la sustitución.
7.  **Solución:** `model.solve()` ejecuta el optimizador.
8.  **Resultados:**
    *   Se verifica el estado (`Optimal`, `Infeasible`, etc.) y se reporta el costo óptimo.
    *   Se muestra un DataFrame `df_resultados` con las asignaciones positivas.
    *   Se realizan **verificaciones detalladas**:
        *   **Cumplimiento de Demanda Granular:** Se itera sobre cada `(d, t_req)` y se comprueba si la suma de horas provistas por asesores calificados cumple la demanda específica.
        *   **Uso de Capacidad:** Se verifica que ninguna prestadora exceda su capacidad total.
        *   **Cumplimiento Límites 70/30:** Se comprueba que las horas asignadas por grupo (básico/experto) en cada prestadora no excedan los límites porcentuales de su capacidad.


---

### **5. Próximos Pasos y Mejoras (Considerando Demanda Granular)**

1.  **Obtener Datos Reales (¡CRUCIAL!):**
    *   **Costos de Viaje:** Reemplazar la matriz simulada con costos reales o estimaciones fiables entre sedes y destinos.
    *   **Demanda Granular:** Sustituir la simulación `demanda_dept_tipo` con datos históricos o proyecciones reales de horas requeridas por departamento Y por nivel de habilidad mínimo necesario.
2.  **Refinar Modelo de Demanda:**
    *   **Requisitos Específicos:** ¿Hay tareas que *solo* puede hacer un T7, o que requieren una habilidad específica no capturada solo por el nivel? Esto podría necesitar dimensiones adicionales en la demanda y variables.
    *   **Sustitución:** Validar la regla de sustitución. ¿Hay penalizaciones de tiempo o eficiencia si un T7 hace trabajo de T1? ¿Es siempre posible/deseable?
3.  **Mejorar Modelo de Costos y Viajes:**
    *   **Viajes Discretos:** Implementar variables enteras para `Num_Viajes[p, t, d]` y relacionarlas con `horas_asignadas` (ej., `horas <= Num_Viajes * Max_Horas_Viaje`). Incluir costos fijos de viaje directamente (`Num_Viajes * Costo_Fijo_Viaje`) en el objetivo. Esto es más complejo pero más preciso.
4.  **Refinar Restricciones de Oferta:**
    *   **Disponibilidad Específica de Asesores:** Añadir restricciones si se conoce el número exacto de asesores de cada tipo en cada prestadora (ej: `Horas_Asignadas_T7_P1 <= Num_Asesores_T7_P1 * Horas_Max_Mes`).
    *   **Flexibilidad 70/30:** Investigar si los límites son estrictos. Si no, modelarlos como objetivos o con penalizaciones (soft constraints).
5.  **Manejo de Infactibilidad y Prioridades:**
    *   **Soft Constraints:** Si satisfacer toda la demanda granular bajo las reglas 70/30 es imposible (infactible), introducir penalizaciones por demanda no satisfecha o por violar ligeramente los límites 70/30, permitiendo al modelo encontrar la "mejor solución posible".
    *   **Priorización:** Si no se puede cumplir toda la demanda, ¿hay departamentos o tipos de demanda prioritarios? Incorporar pesos o prioridades.
6.  **Variables Enteras:** Considerar usar `LpInteger` para `horas_asignadas` si se requiere facturación por horas completas o para el modelo de viajes discretos. Aumenta el tiempo de cómputo.
7.  **Análisis de Sensibilidad:** Una vez con datos reales, explorar cómo cambios en costos, demanda, o capacidades afectan las asignaciones y el costo total. Evaluar escenarios "what-if".
8.  **Horizonte Temporal:** Considerar si un modelo mensual es suficiente o si se requiere una planificación semanal o incluso diaria para capturar variaciones más rápidas.